In [ ]:
# Create a folder in your home directory and change directory here
import os
os.chdir(r"enter path here")

# Create bloblist function


def blobList():
    # Import blob client for connection
    from azure.storage.blob import BlobServiceClient as bsc

    print("\nWelcome 👋🏽, \n\tthis is a script to load a batch of blobs to MongoDB as JSON documents.\n")

    global container
    container = str(input("Enter your container name 🫙 : "))
    global path
    path = str(input("\nEnter path to the blobs 🫧 : "))
    global accountUrl
    # Enter your account url here from endpoint page in azure portal.
    accountUrl = "enter account url here"
    global accessKey
    # Enter your access key here from the access keys page in azure portal.
    accessKey = "enter access key here"
    # Initializing connection to storage account to list the blobs in a folder
    blob_service_client_instance = bsc(account_url=accountUrl,
                                       credential=accessKey)
    container_client = blob_service_client_instance.get_container_client(
        container)
    blob_list = container_client.list_blobs(
        name_starts_with=path)
    # Creating the list
    global blobarray
    blobarray = []
    for blob in blob_list:
        string = str(blob.name).replace(path, "")
        blobarray.append(string)

    print("\n", [f"{blobarray.index(row)+1}. {row}" for row in blobarray])


# Function call to display blobs in folder
blobList()

# Create azureToMongo function


def azureToMongo():

    import xml.etree.ElementTree as ET  # For parsing binary string to XML
    import xmltodict  # For parsing XML to dictionary
    # For connection to storage account
    from azure.storage.blob import BlobServiceClient as bsc
    from pymongo import MongoClient as mc  # For exporting dict to MongoDB

    fromm = int(
        input("\nEnter from (index of blob you want to start from the above list): "))
    to = int(input(f"\nfrom {blobarray[fromm-1]} to..? : "))
    print(f"You've selected {blobarray[fromm-1]} to {blobarray[to-1]} 🗂️")
    # looping each selected blobs to MongoDB
    blob = blobarray[fromm-1:to]
    for row in blob:
        localname = row
        blob_service_client_instance = bsc(account_url=accountUrl,
                                           credential=accessKey)
        blob_client_instance = blob_service_client_instance.get_blob_client(
            container, f"{path}{row}", snapshot=None)
        with open(localname, "wb") as my_blob:
            blob_data = blob_client_instance.download_blob()
            blob_data.readinto(my_blob)
        # Converting binary string to XML and converting to dictionary
        parsed = ET.parse(localname)
        root = parsed.getroot()
        xmlstr = ET.tostring(root, method='xml')
        data = xmltodict.parse(xmlstr)
        # Connecting to target collection in MongoDB and inserting each file
        client = mc(
            "enter mongodb address here")
        collection = client["enter database"]["enter collection"]
        collection.insert_one(data)
        client.close()

    print("\nSuccessful! 🤩\n")


# Function call to insert XML files to MongoDB
azureToMongo()
